# DSI Team Starting Point

### Background

DSI Phase I:
- Cleaned and Analyzed Historical CFPB Agreement Data looking to understand offered APRs
    - Team found initial analysis lacking in meaning due to the nature of the APR values reported in the agreements
        - The Reported APRs were listed as a "middle point" across customer segments per plan per agreement.
        - This ultiamtely meant that the APR value was already a range or a value from a range rather than the discreet values themselves.
    - Later found out from data stewards that the historical data is non-verifiable, meaning the reported APRs were hard to relate to the actual offerings at that time.
    - Data Teams Presentations:
        - <a href="https://github.com/chrisJoyceDS/top_sprint_data/blob/main/code/CFPB_Data.ipynb"> Jupyter Notebook Repo </a>
        - <a href="https://docs.google.com/presentation/d/16vBsI7b5Fw_0tF1UQK4qW2dn32v66PwcC3VZ959mx4c/edit#slide=id.p"> Presentation to the Team </a>
        - <a href="https://docs.google.com/presentation/d/18uy5qkw0zQTg-SA-ceXDKOI2dCI4RLN-IjSQXfIwCIM/edit#slide=id.p"> Presentation to the CFPB </a>

DSI Phase II:
- After having it confirmed that the historical data was innacurate and far from reality, the team learned of and used the updated dataset provided by the CFPB
- This data was from July 2022 to December 2022 and had a variety of updated information with respect to the historical data
- Unfortunately there is no data dictionary for this dataset, and we will have to interpet their values based on the column descriptions
- for the sake of the team, we left most of the column names intact, and update a few where we thought it was best/relevant
- Please follow us along or Select "Jump to Clustering" below:

[Jump to Clustering](#Clustering)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import cm
import seaborn as sns
import re
import missingno as msno

# imputing
from sklearn.impute import KNNImputer
# preprocessing
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import make_column_transformer
#clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

- The dataset data doesn't fall on the usual A0 cell, we needed to adjust the pandas reader

In [ ]:
columns_to_read = 'B:FO'

In [ ]:
df = pd.read_excel("../data/cfpb_updated_tccp.xlsx",skiprows=9,usecols=columns_to_read)

In [ ]:
df.head()

In [ ]:
df.shape

- Key differences between this data set and the historical data set
    - This is not timeseries, it is each institution's plan data as of December 31 of the participants in the survey
    - Phase I dataset was (rows:9556,columns:12)
    - more dense of a dataset
- let's check out the null landscape below

In [ ]:
msno.matrix(df.sample(250))

- What you are seeing is a graphical representation of the data within the new data set:
    - Black means data
    - White means no data
- With more fields collected, comes the even possibility of not collecting it.
- Below is the start of our initial clean process. Because of the look of the dataset above, and the sheer amount of initial features we have (170) we are going to make some swift cuts

In [ ]:
df.info()

In [ ]:
# Calculate the percentage of null values in each column
null_percentage = (df.isnull().sum() / len(df)) * 100

In [ ]:
type(null_percentage)

In [ ]:
null_percentage.shape

In [ ]:
# set threshold
threshold = 90

In [ ]:
# Create a list of column names to drop where the null percentage is greater than or equal to the threshold
columns_to_drop = null_percentage[null_percentage >= threshold].index.tolist()

In [ ]:
len(columns_to_drop)

In [ ]:
# drop the columns
df.drop(columns=columns_to_drop,inplace=True)

In [ ]:
df.shape

- Above we:
    - Calculated the null percentage for each column of the 170
    - set a threshold of 90 percent null as the cut off
    - removed all columns that met that threshould and above
    - resulting in dropping 68 columns
- You can see the impact below, and it's much better, but we still have 102 features, which means we will be greedier in our selection

In [ ]:
msno.matrix(df.sample(250))

In [ ]:
df.head()

In [ ]:
# get remaining column data types
col_types = df.dtypes

In [ ]:
# separate out numerical and object columns
num_cols = col_types[col_types != 'object'].index.tolist()
obj_cols = col_types[col_types == 'object'].index.tolist()

In [ ]:
df[num_cols].shape

In [ ]:
df[obj_cols].shape

In [ ]:
df[num_cols].isnull().sum().sum(), df[obj_cols].isnull().sum().sum()

In [ ]:
# re-Calculate the percentage of null values in each column
null_percentage = (df.isnull().sum() / len(df)) * 100

In [ ]:
null_percentage.sort_values(ascending=False)[null_percentage >= 59]

In [ ]:
# additional columns to drop
cols_to_drop = null_percentage[null_percentage >= 60].index.tolist()

In [ ]:
len(cols_to_drop)

In [ ]:
# drop additional columns
# drop the columns
df.drop(columns=cols_to_drop,inplace=True)

In [ ]:
df.info()

- Same exercise, new threshold, this time anything that was greater or equal to 59% null
- Now we have 68 Columns (5.5 times the features we had with historical btw) with a much better look of data below

In [ ]:
msno.matrix(df.sample(250))

In [ ]:
df.columns.shape

- Something you will have noticed, if you read our initial analysis, is that there are less visualizations for this data set
- There will be more going forward, but for now we have prioritized creating aggregate cards for the Software Engineering Team to be able to start building
- From this point we will be doing a mix of cleaning, feature engineering, preprocessing, and clustering

---
## MVP Goal

- Create Aggregate cards based on the targeted credit tiers 
- Capture APR, Grace Periods, Late Fees, Rewards, and Annual Fees

In [ ]:
df["Targeted Credit Tiers"] = df["Targeted Credit Tiers"].apply(lambda x: x.lower())

In [ ]:
df["Targeted Credit Tiers"].value_counts()

- split the strings on the ';'
- for the non technical this just makes it easier for us to interact with
- Goes from:
    - good credit (credit scores from 620 to 719); great credit (credit score of 720 or greater) 
    - to
    - ['good credit (credit scores from 620 to 719)',
 'great credit (credit score of 720 or greater)']

In [ ]:
df["Targeted Credit Tiers"] = df["Targeted Credit Tiers"].str.split("; ")

- Create three columns that align with the credit score group.
- if the word is found in a given list of options for a specific group, report True, else False 

In [ ]:
# Create Separate Columns for each bucket
df['Poor_Fair_Group'] = df['Targeted Credit Tiers'].apply(lambda x: any(re.search(r'poor', s) for s in x))
df['Good_Credit_Group'] = df['Targeted Credit Tiers'].apply(lambda x: any(re.search(r'good', s) for s in x))
df['Great_Credit_Group'] = df['Targeted Credit Tiers'].apply(lambda x: any(re.search(r'great', s) for s in x))

In [ ]:
df[['Poor_Fair_Group','Good_Credit_Group','Great_Credit_Group']]

In [ ]:
# convert boolean into numerical values
df['Poor_Fair_Group'] = df['Poor_Fair_Group'].astype(int)
df['Good_Credit_Group'] = df['Good_Credit_Group'].astype(int)
df['Great_Credit_Group'] = df['Great_Credit_Group'].astype(int)

In [ ]:
df[['Poor_Fair_Group','Good_Credit_Group','Great_Credit_Group']].head()

In [ ]:
df.columns

In [ ]:
df.head()

Immediately, some of these columns are aparently redundant. For instance:

In [ ]:
df[['Purchase APR Offered?', 'Purchase APR Vary by Balance']]

Whether an APR is offered is already represented in `Purchase APR Vary by Balance` because in cases where an APR is not offered, there will be a NaN. That said, I am going to have to impute these NaNs before I can cluster, so this relationship will be lost, but would be retained by `Purchase APR Offered?` if I decide to keep this feature.

Some other features I assume won't be useful:

In [ ]:
df[['Website for Consumer','Telephone Number for Consumers']]

Is there a real relationship between credit score and the URL/phone number of the company offering a credit card? Probably not. I'll drop these features.

In [ ]:
df.drop(columns = ['Website for Consumer','Telephone Number for Consumers'], inplace = True)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df[['Good Credit', 'Good_Credit_Group']]

I'm not sure what's going on here^ </br>
Good_Credit_Group was a feature engineered by Chris. Good Credit appears to be one of the original features from the dataset.

In [ ]:
df[['Other Fees', 'Additional Fees']]

In [ ]:
df[df['Other Fees'] == 'No'][['Other Fees', 'Additional Fees']]

It appears that "Additional Fees" is dependent on "Other Fees", where the former being non-null depends on the latter being non-null.

As with the APR features examined above, this means these two features are redundant, but I'm not sure if I want to get rid of one of them as the original relationship will be lost when I impute the nulls.

In [ ]:
df[['Late Fees?', 'Late Fee Types', 'Amount (Dollars) - Late Payment Fee',
       'Late Fee Six Month Billing Cycle', 'Late Fee Policy Details']]

In [ ]:
df['Late Fee Types'].unique().tolist()

There is clearly something messed up with this feature. It appears to contain the text data used to collect responses i.e. these are the prompts used to get banks to imput info about their late fees. Probably, I'll just drop this feature as I don't know what else to do with it and it doesn't appear to contain useful information.

In [ ]:
df.drop(columns = ['Late Fee Types'], inplace = True)

In [ ]:
df.columns

In [ ]:
df[['Grace Period Offered?', 'Balance Transfer Grace Period', 'Grace Period']]

In [ ]:
df[df['Grace Period Offered?'] == 'No'][['Grace Period Offered?', 'Balance Transfer Grace Period', 'Grace Period']]

So, the balance transfer grace period is not a subset of the overall grace period offered. In row 581 above, there is no grace period offered according to the first column, but aparently this does not stop there from being a balance transfer grace period. Based on this, I don't understand what the first column is refering to. The third column does not appear to be in reference to the balance transfer grace period as there is no information (length of grace period) specified for 581 either. Although all of this is assuming the data is complete and accurate.

In [ ]:
df[['Over Limit Fees?', 'Other Fees', 'Additional Fees', 'Other Fee Name',
       'Other Fee Amount', 'Other Fee Explanation', 'Other Fee Name.1',
       'Other Fee Amount.1', 'Other Fee Explanation.1']]

In [ ]:
df[['Other Fees', 'Other Fee Name']]

As mentioned several times, this dataset is full of redundant features where the first column is a boolean, and the second column actually contains the data, but only the first column is 'Yes'. With these, I would like to get rid of as many redundant features as I can, and I'm realizing that I can probably drop the first column by setting NaNs in the second column to either 0 or str type "None" depending on the dtype. This way, I retain the original meaning contained in the first column, I have a straightforward fill technique and I can drop many redundant features.

In [ ]:
df['Other Fee Name'].replace(np.NaN, 'None', inplace = True)

In [ ]:
df['Other Fee Name']

As examined before, 'Additional Fees' is a subset of 'Other Fees'. 'Additional Fees' can only be 'Yes' when 'Other Fees' is 'Yes'. 

In [ ]:
df[['Other Fees', 'Additional Fees']]

Based on this, it seems reasonable to assume that 'NaN' values in 'Additional Fees' can be filled with  'No' and still retain the original meaning. I am assuming that 'NaN' values in 'Additional Fees' are a product of it being a subset of 'Other Fees'. If I drop 'Other Fees' and fill NaNs in 'Additional Fees' with 'No', the original relationship seen in the table above will be lost, but I assume that this relationship was not valid to begin with, and NaNs in 'Additional Fees' are actually supposed to be 'No'.

In [ ]:
df['Additional Fees'].replace(np.NaN, 'No', inplace = True)

In [ ]:
df['Additional Fees'].describe()

**Examining the two features I just modified**

In [ ]:
df[['Other Fees', 'Other Fee Name', 'Additional Fees']]

Now we can see the product of my imputation decision. The first two columns have a clear relationship, but Additional Fees is only Yes in cases where Other Fees is Yes. Even then, Additional Fees is only Yes in a fraction of cases. I think this makes sense - there are a variety of fees in the dataset related to balance transfer, purchase transaction, etc. 

Other fees captures those fees specific to certain banks, with the name contained in Other Fee Name. Additional Fees is an even smaller subset of cases not recorded in Other Fees for some reason. 

Ideally, I would like to combine Additional Fees and Other Fees into one feature. I could have 0 be No, 1 be Other Fees == Yes and 2 be Other Fees AND Additional Fees == Yes.

In [ ]:
df.columns

The issue is that there is no clearly labeled Additional Fee Amount feaure, and capturing the actual numeric amount of the fee would be better than simple OHE. Maybe Other Fee Amount.1 contains info for Additional Fees:

In [ ]:
df[['Other Fee Amount', 'Other Fee Amount.1', 'Additional Fees']]

In [ ]:
df[df['Additional Fees'] == 'Yes'][['Other Fee Amount.1', 'Additional Fees']]

In [ ]:
df[df['Additional Fees'] == 'No'][['Other Fee Amount.1', 'Additional Fees']]

Based on the two tables above, it appears that Other Fee Amount.1 contains the cash amount of Additional Fees. Excellent, I can now combine the two Other Fee Amount features into one, rendering several features redundant.

Before doing this though, I need to convert NaNs in Other Fee Amount.1 to 0 so that this addition works properly.

In [ ]:
df['Other Fee Amount.1'].replace(np.NaN, 0, inplace = True)

In [ ]:
df['other_fee_amount'] = df['Other Fee Amount'] + df['Other Fee Amount.1']

In [ ]:
df[['other_fee_amount', 'Other Fee Amount', 'Other Fee Amount.1']]

Now that I think about it, it's not really relevant whether or not a fee exists if its value is 0. I thought this would help capture some significant information in the features I'm aiming to drop, it it doesn't. I'll simply convert NaNs in my new feature to 0:

In [ ]:
df['other_fee_amount'].replace(np.NaN, 0, inplace = True)

In [ ]:
df[['other_fee_amount', 'Other Fee Amount', 'Other Fee Amount.1']]

There are a lot of other features related to other fees regarding their name, explaination, etc, but I don't think these will help our model. All of the relevant information is now condensed into other_fee_amount. I'll drop the unhelpful features:

In [ ]:
df.shape

In [ ]:
df.drop(columns = ['Other Fees', 'Additional Fees', 'Other Fee Name',
       'Other Fee Amount', 'Other Fee Explanation', 'Other Fee Name.1',
       'Other Fee Amount.1', 'Other Fee Explanation.1'], inplace = True)

In [ ]:
df.shape

**Revisiting other redundant features**

In [ ]:
df.columns

In [ ]:
df[['Grace Period Offered?', 'Balance Transfer Grace Period', 'Grace Period']]

In [ ]:
df[df['Grace Period Offered?'] == 'No'][['Grace Period Offered?','Grace Period']]

In [ ]:
df[df['Grace Period Offered?'] == 'Yes'][['Grace Period Offered?','Grace Period']]

Grace Period Offered is redundant because NaNs in Grace Period simply correspond to a No in the former column. Balance Transfer Grace Period must be a subset type of grace period that only some cards offer. I'll simply drop the first column and convert NaNs in the third to 0.

In [ ]:
df['Grace Period'].replace(np.NaN, 0, inplace = True)

In [ ]:
df.drop(columns = ['Grace Period Offered?'], inplace = True)

**Removing features I don't think will help the model**

In [ ]:
df.head()

Contact Information Types, Report Date and Created Date don't appear to be useful

In [ ]:
df[df.columns[11:-11]]

In [ ]:
df.columns

**Feature engineering late fee info**

In [ ]:
df[['Late Fees?',
       'Amount (Dollars) - Late Payment Fee',
       'Late Fee Six Month Billing Cycle', 'Late Fee Policy Details']]

In [ ]:
df[df['Late Fee Policy Details'].isna() == False][['Late Fees?',
       'Amount (Dollars) - Late Payment Fee',
       'Late Fee Six Month Billing Cycle', 'Late Fee Policy Details']]

In [ ]:
len(df[df['Late Fee Policy Details'].isna() == False]['Late Fee Policy Details'].unique())

In [ ]:
df[df['Late Fee Policy Details'].isna() == False]['Late Fee Policy Details'][6]

In [ ]:
df[df['Late Fee Policy Details'].isna() == False]['Late Fee Policy Details'][4]

In [ ]:
df[df['Late Fee Policy Details'].isna() == False]['Late Fee Policy Details'].unique()[5]

In [ ]:
df[df['Late Fee Policy Details'].isna() == False]['Late Fee Policy Details'].unique()[5]

In [ ]:
df[(df['Late Fee Policy Details'].notna()) 
                & (df['Amount (Dollars) - Late Payment Fee'].notna())][[
                'Amount (Dollars) - Late Payment Fee',
                'Late Fee Policy Details']]

Here are some examples where both the late fee amount and the details are filled out. I can use this as a reference for how to impute missing late fee amounts using the details feature. In general, there's a dollar specified in the details which is identical to the late fee amount. Here is an exception:

In [ ]:
df[(df['Late Fee Policy Details'].notna()) 
                & (df['Amount (Dollars) - Late Payment Fee'].notna())][[
                'Amount (Dollars) - Late Payment Fee',
                'Late Fee Policy Details']].loc[78]

The details state "We charge up to $35.00." There would be no way to know that the late fee is assessed at `$29` in this case.

In [ ]:
df[(df['Late Fee Policy Details'].notna()) 
                & (df['Amount (Dollars) - Late Payment Fee'].notna())]['Late Fee Policy Details'][62]

There are multiple dollar amounts listed here. The lower number is the original late fee, the second is an additional fee assessed after six months of non-payment.

In [ ]:
df[df['Late Fee Policy Details'].isna() == False][['Late Fees?',
       'Amount (Dollars) - Late Payment Fee',
       'Late Fee Six Month Billing Cycle', 'Late Fee Policy Details']].loc[62]

I'll try to extract the dollar amount for the late fees from the description. If there's more than one dollar sign in the description, I'll set the higher amount to be the six month fee, assuming the amounts are different.

In [ ]:
df[df['Late Fee Policy Details'].isna() == False]['Late Fee Policy Details'][4]

This approach won't really work in this instance. ^ The $27 appears to be assessed immediately, not after a period of 6 months. Still, I think the general idea is the same regardless of the time period, as there are multiple tiers of fees being assessed depending on the borrower's debt status. I'll manually fix this error, and accept that there may be similar errors for now. I'll try another approach with kNN imputation of this feature once the dataset is complete.

Finding the index for relevant cases where there is a late fee needing imputation and there are policy details I can use to impute:

In [ ]:
policy_index = df[(df['Amount (Dollars) - Late Payment Fee'].isna() == True)
   & (df['Late Fee Policy Details'].notna())].index

In [ ]:
df.loc[policy_index][[
                'Amount (Dollars) - Late Payment Fee',
                'Late Fee Policy Details']]

In [ ]:
def fee_imputer():
    fees_df = pd.DataFrame(index=range(len(df)), columns=['fee', 'late_fee'])
    for i in range(len(df)):
        if i in policy_index:
            policy = df['Late Fee Policy Details'][i].split('$')[1:]
            temp_df = pd.DataFrame(index=[i], columns=['fee', 'late_fee'])

            if len(policy) == 2:
                policy[0] = policy[0].replace(',', '').strip('.').split('.')[0].split(' ')[0]
                policy[1] = policy[1].replace(',', '').strip('.').split('.')[0].split(' ')[0]

                temp_df['fee'] = int(policy[0])
                temp_df['late_fee'] = int(policy[1])

                if policy[0] == policy[1]:
                    temp_df['late_fee'] = 0

                fees_df.loc[i] = temp_df.loc[i]

            elif len(policy) == 1:
                policy[0] = policy[0].replace(',', '').strip('.').split('.')[0].split(' ')[0]
                temp_df['fee'] = int(policy[0])
                temp_df['late_fee'] = 0

                fees_df.loc[i] = temp_df.loc[i]
        else: 
            fees_df.loc[i] = [0, 0]
    return fees_df

result = fee_imputer()
result

In [ ]:
result.loc[policy_index]

In general this has worked, but there are some errors due to some numbers lacking dollar signs. Splitting using numerics won't work because there are many percentages throughout. I'll simply have to impute these another way.

In [ ]:
df['Late Fee Policy Details'][599]

In [ ]:
result[result['fee'].isna() == True]

In [ ]:
df['Late Fee Policy Details'][58]

^ Again, there is no clear way to account for this using my function. I'll kNN impute once I clean all the other features.

I'll add in my new values by replacing all the NaNs in the existing columns with 0, then I'll add the new values to them. The new NaNs will cause the 0s to be set to NaNs.

In [ ]:
df[['Amount (Dollars) - Late Payment Fee','Late Fee Six Month Billing Cycle']]

In [ ]:
columns_to_fill = ['Amount (Dollars) - Late Payment Fee', 'Late Fee Six Month Billing Cycle']
df.loc[:, columns_to_fill] = df[columns_to_fill].fillna(0)

In [ ]:
df[['Amount (Dollars) - Late Payment Fee','Late Fee Six Month Billing Cycle']]

In [ ]:
df['Amount (Dollars) - Late Payment Fee'] + result['fee']

In [ ]:
df['fee'] = df['Amount (Dollars) - Late Payment Fee'] + result['fee']
df['late_fee'] = df['Late Fee Six Month Billing Cycle'] + result['late_fee']

The nulls have been maintained:

In [ ]:
df[df['fee'].isna() == True][['fee','late_fee']]

In [ ]:
result[result['fee'].isna() == True]

In [ ]:
df.columns

In [ ]:
df.drop(columns = ['Late Fees?',
                   'Amount (Dollars) - Late Payment Fee',
                   'Late Fee Six Month Billing Cycle', 
                   'Late Fee Policy Details'], inplace = True)

**Cleaning First 10 Features**

In [ ]:
df.iloc[:5, 0:11]

In [ ]:
df.iloc[:, 0:11].isnull().sum()

**Examing APR features:**

In [ ]:
df[['Purchase APR Offered?', 'Purchase APR Vary by Balance',
       'Purchase APR Index', 'Variable Rate Index', 'Index']]

In [ ]:
df['Variable Rate Index'].unique()

In [ ]:
df[df['Variable Rate Index'] == 'Six-month T-bill'][['Variable Rate Index', 'Index']]

In [ ]:
df[df['Variable Rate Index'] == 'Prime'][['Variable Rate Index', 'Index']]

Variable Rate Index does not appear to contain any information not already represented in Index.

In [ ]:
df[df['Variable Rate Index'].isna() == True][['Variable Rate Index', 'Index']]

The only difference is that nulls in Variable are not present in Index. This is a subtle difference, but I would prefer to preserve this relationship for now. I'll fill in nulls rather than dropping the feature:

In [ ]:
df['Variable Rate Index'].isna().sum()

In [ ]:
df['Variable Rate Index'].replace(np.NaN, 'None', inplace = True)

In [ ]:
df[['Variable Rate Index', 'Index']]

In [ ]:
df.iloc[:, 0:11].isnull().sum()

In [ ]:
df[['Purchase APR Offered?', 'Purchase APR Vary by Balance',
       'Purchase APR Index', 'Variable Rate Index', 'Index']]

In [ ]:
df[['Purchase APR Offered?','Purchase APR Vary by Balance']]

In [ ]:
df[df['Purchase APR Offered?'] == 'No'][['Purchase APR Offered?','Purchase APR Vary by Balance']]

As with so many of these features, the second one is dependent on the first. Again, I'll simply fill in NaNs with 'None' to preserve the subset relationship:

In [ ]:
df['Purchase APR Vary by Balance'].replace(np.NaN, 'None', inplace = True)

In [ ]:
df['Purchase APR Vary by Balance'].isna().sum()

In [ ]:
# Similar relationship with Purchase APR Index
df[['Purchase APR Offered?','Purchase APR Index']]

Same relationship. I'll maintain it by filling with 'None':

In [ ]:
df['Purchase APR Index'].isna().sum()

In [ ]:
df['Purchase APR Index'].replace(np.NaN, 'None', inplace = True)

In [ ]:
df['Purchase APR Index'].isna().sum()

In [ ]:
df.iloc[:, 0:11].isnull().sum()

In [ ]:
df["Targeted Credit Tiers"].apply(lambda x: x.lower())

**Cleaning Features 11-21**

In [ ]:
df.iloc[:, 11:22].isnull().sum()

In [ ]:
df.iloc[:5, 11:22]